In [6]:
import numpy
from matplotlib import pyplot as plt

import keras
from vis.visualization import visualize_activation
from vis.utils import utils
from keras import activations
%matplotlib inline
plt.rcParams['figure.figsize'] = (18, 6)

In [13]:
        from keras.layers import Input, Dense, UpSampling2D, Conv2D, Reshape, BatchNormalization, Activation
        from keras.models import Model
        latent_dim = 100
        ini_size = 7
        
        x_input = Input(shape=(latent_dim,), name='g_input0')
        x = Dense(ini_size * ini_size * 128, name='g_dense1')(x_input)
        x = BatchNormalization(name='g_bn1')(x)
        x = Activation('relu', name='g_ac1')(x)

        x = Reshape((ini_size, ini_size, 128), name='g_reshape1')(x)

        x = UpSampling2D((2, 2), name='g_up2')(x)
        x = Conv2D(128, kernel_size=5, strides=1, padding='same', name='g_conv2')(x)
        x = BatchNormalization(name='g_bn2')(x)
        x = Activation('relu', name='g_ac2')(x)
        x = Conv2D(64, kernel_size=5, strides=1, padding='same', name='g_conv3')(x)
        x = BatchNormalization(name='g_bn3')(x)
        x = Activation('relu', name='g_ac3')(x)
        x = UpSampling2D((2, 2), name='g_up3')(x)
        x = Conv2D(32, kernel_size=5, strides=1, padding='same', name='g_conv4')(x)
        x = BatchNormalization(name='g_bn4')(x)
        x = Activation('relu', name='g_ac4')(x)

        x = Conv2D(3, kernel_size=5, strides=1, padding='same', name='g_conv5')(x)
        x = BatchNormalization(name='g_bn5')(x)
        x = Activation('relu', name='g_ac5')(x)

        x = Conv2D(1, kernel_size=3, strides=1, padding='same', name='g_conv6')(x)
        x_output = Activation('sigmoid', name='g_ac6')(x)

        model = Model(inputs=x_input, outputs=x_output, name='generator')
        model.summary()
        model.load_weights('weights/generator.hdf5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
g_input0 (InputLayer)        (None, 100)               0         
_________________________________________________________________
g_dense1 (Dense)             (None, 6272)              633472    
_________________________________________________________________
g_bn1 (BatchNormalization)   (None, 6272)              25088     
_________________________________________________________________
g_ac1 (Activation)           (None, 6272)              0         
_________________________________________________________________
g_reshape1 (Reshape)         (None, 7, 7, 128)         0         
_________________________________________________________________
g_up2 (UpSampling2D)         (None, 14, 14, 128)       0         
_________________________________________________________________
g_conv2 (Conv2D)             (None, 14, 14, 128)       409728    
__________

ValueError: You are trying to load a weight file containing 9 layers into a model with 11 layers.

In [12]:
# Utility to search for layer index by name. 
# Alternatively we can specify this as -1 since it corresponds to the last layer.
layer_idx = utils.find_layer_idx(model, 'g_conv6')

# Swap softmax with linear
model.layers[layer_idx].activation = activations.linear
model = utils.apply_modifications(model)

# This is the output node we want to maximize.
filter_idx = 0
img = visualize_activation(model, layer_idx, filter_indices=filter_idx)
plt.imshow(img[..., 0])

/anaconda3/envs/pycharm/lib/python3.6/site-packages/keras/models.py:282: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


ValueError: Tried to convert 'x' to a tensor and failed. Error: None values not supported.